<a href="https://colab.research.google.com/github/eveugla/333/blob/main/%D0%94%D0%B8%D0%BF%D0%BB%D0%BE%D0%BC_%D1%84%D0%B8%D0%BD%D0%B0%D0%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://parl.ai/downloads/personachat/personachat.tgz
!tar -xvzf personachat.tgz

--2023-09-30 11:10:36--  https://parl.ai/downloads/personachat/personachat.tgz
Resolving parl.ai (parl.ai)... 13.33.88.61, 13.33.88.129, 13.33.88.78, ...
Connecting to parl.ai (parl.ai)|13.33.88.61|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://dl.fbaipublicfiles.com/parlai/personachat/personachat.tgz [following]
--2023-09-30 11:10:36--  https://dl.fbaipublicfiles.com/parlai/personachat/personachat.tgz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.251.114, 52.84.251.27, 52.84.251.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.251.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223221886 (213M) [application/x-tar]
Saving to: ‘personachat.tgz’

personachat.tgz     100%[===================>] 212.88M  13.5MB/s    in 33s     

2023-09-30 11:11:11 (6.43 MB/s) - ‘personachat.tgz’ saved [223221886/223221886]

tar: Ignoring unknown extended header keyword 'LIBARCHI

In [2]:
import gensim
from gensim.models import Word2Vec
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KDTree
import os
import json

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Загружаем и обрабатываем данные
with open('personachat/train_both_original.txt', 'r') as f:
    lines = f.readlines()

In [5]:
# Обработка данных
persona_info = []
conversations = []
for line in lines:
    if "your persona:" in line:
        persona_info.append(line.replace("your persona:", "").strip())
    else:
        tokens = line.split("\t")
        if len(tokens) >= 2:
            if persona_info:
                conversations.append((persona_info, tokens[0], tokens[1]))
                persona_info = []

qa_pairs = []
for conversation in conversations:
    persona, question, answer = conversation
    combined_question = ' '.join(persona) + ' ' + question
    qa_pairs.append((combined_question, answer))

In [6]:
# Создаем словарь слов
all_words = set()
for pair in qa_pairs:
    question = [word.lower() for word in word_tokenize(pair[0]) if word.isalpha() and word.lower() not in stop_words]
    answer = [word.lower() for word in word_tokenize(pair[1]) if word.isalpha() and word.lower() not in stop_words]
    all_words.update(question)
    all_words.update(answer)

In [7]:
import pickle
# Сохраняем вопросы и ответы
questions = [pair[0] for pair in qa_pairs]
answers = [pair[1] for pair in qa_pairs]

# Сохраняем вопросы и ответы в файлы
with open('questions.pkl', 'wb') as f:
    pickle.dump(questions, f)

with open('answers.pkl', 'wb') as f:
    pickle.dump(answers, f)

In [8]:
import multiprocessing

# Word2Vec
sentences = [word_tokenize(pair[0]) for pair in qa_pairs] + [word_tokenize(pair[1]) for pair in qa_pairs]
model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=multiprocessing.cpu_count())
model_w2v.save("word2vec.model")

# Предобработка данных
def text_to_vector(text):
    words = word_tokenize(text)
    vector = np.zeros(model_w2v.vector_size)
    count = 0
    for word in words:
        if word in model_w2v.wv:
            vector += model_w2v.wv[word]
            count += 1
    if count > 0:
        vector /= count
    return vector

train_x_vec = np.array([text_to_vector(pair[0]) for pair in qa_pairs])
train_y_vec = np.array([text_to_vector(pair[1]) for pair in qa_pairs])

train_x, val_x, train_y, val_y = train_test_split(train_x_vec, train_y_vec, test_size=0.2, random_state=42)

In [9]:
# Создаем модель
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=model_w2v.vector_size))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(model_w2v.vector_size, activation='linear'))

model.compile(optimizer='adam', loss='mse')
hist = model.fit(train_x, train_y, epochs=200, batch_size=64, validation_data=(val_x, val_y))
model.save('dense_model.h5')

Epoch 1/200
112/112 [==============================] - 9s 9ms/step - loss: 0.1177 - val_loss: 0.0840
Epoch 2/200
112/112 [==============================] - 1s 7ms/step - loss: 0.0863 - val_loss: 0.0795
Epoch 3/200
112/112 [==============================] - 1s 6ms/step - loss: 0.0830 - val_loss: 0.0795
Epoch 4/200
112/112 [==============================] - 1s 6ms/step - loss: 0.0817 - val_loss: 0.0786
Epoch 5/200
112/112 [==============================] - 1s 5ms/step - loss: 0.0807 - val_loss: 0.0774
Epoch 6/200
112/112 [==============================] - 0s 4ms/step - loss: 0.0799 - val_loss: 0.0778
Epoch 7/200
112/112 [==============================] - 0s 4ms/step - loss: 0.0797 - val_loss: 0.0771
Epoch 8/200
112/112 [==============================] - 0s 4ms/step - loss: 0.0792 - val_loss: 0.0774
Epoch 9/200
112/112 [==============================] - 0s 4ms/step - loss: 0.0787 - val_loss: 0.0764
Epoch 10/200
112/112 [==============================] - 0s 4ms/step - loss: 0.0786 - val_lo

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
# Создание массива векторов ответов
answer_vectors = [text_to_vector(pair[1]) for pair in qa_pairs]
answer_texts = [pair[1] for pair in qa_pairs]

# Создание структуры данных KDTree для быстрого поиска ближайших векторов
tree = KDTree(answer_vectors)

def get_closest_response(vector_representation):
    _, ind = tree.query([vector_representation], k=1)
    return answer_texts[ind[0][0]]

def getResponse(pattern):
    vector_representation = text_to_sequence(pattern)
    vector_representation = np.expand_dims(vector_representation, axis=0)
    predictions = model.predict(vector_representation)[0]
    response_sentence = get_closest_response(predictions)
    return response_sentence

def text_to_sequence(text):
    words = [word for word in word_tokenize(text.lower()) if word in model_w2v.wv]
    sequence = [model_w2v.wv[word] for word in words]
    return np.mean(sequence, axis=0) if words else np.zeros(model_w2v.vector_size)

def simulate_chatbot_response(sentence):
    response = getResponse(sentence)
    print(f"Input Sentence: {sentence}")
    print(f"Chatbot Response: {response}")

In [12]:
# Тестирование чатбота
simulate_chatbot_response("What's up?")

1/1 [==============================] - 0s 88ms/step
Input Sentence: What's up?
Chatbot Response: i am great , i just got the kids down for a nap . how are you ?


In [14]:
simulate_chatbot_response("Im sad.")

1/1 [==============================] - 0s 22ms/step
Input Sentence: Im sad.
Chatbot Response: hi ! are you a dr , by chance ? i need someone to see my toes .
